In [ ]:
!git clone https://github.com/Acquil/deep-read.git

Cloning into 'deep-read'...
remote: Enumerating objects: 79, done.
remote: Total 79 (delta 0), reused 0 (delta 0), pack-reused 79
Unpacking objects: 100% (79/79), done.


In [ ]:
!unzip /content/deep-read/data/training/CNN/slides/images_rendered_from_powerpoint.zip

In [ ]:
!unzip "/content/deep-read/data/training/CNN/video conference screens/video conference screens 1.zip"
!unzip "/content/deep-read/data/training/CNN/video conference screens/video conference screens 2.zip"
!unzip "/content/deep-read/data/training/CNN/video conference screens/video conference screens.zip"

In [ ]:
!mkdir training_data
!mv /content/images_rendered_from_powerpoint /content/training_data

In [40]:
!mkdir /content/training_data/screens

In [ ]:
!rm -r /content/training_data/screens

In [41]:
import os
folder_list = ['/content/video conference screens/',
          '/content/video conference screens 1/',
          '/content/video conference screens 2/']

for folder in folder_list:
  files = os.listdir(folder)
  for source in files:
    os.rename(folder+source, '/content/training_data/screens/'+source) 

In [42]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
from PIL import Image 
from google.colab import files

In [43]:
base_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(299,299,3)
)
base_model.trainable = False

94773248/94765736 [==============================] - 1s 0us/step


In [51]:
imagedatagen = ImageDataGenerator(rescale=1/255,
                                  validation_split=0.2
                            )
train_generator = imagedatagen.flow_from_directory(
                            '/content/training_data',
                            class_mode="binary",
                            target_size=(299,299),
                            batch_size=32,
                            subset='training')
validation_generator = imagedatagen.flow_from_directory(
                            '/content/training_data',
                            class_mode="binary",
                            target_size=(299,299),
                            batch_size=32,
                            subset='validation')

Found 1821 images belonging to 2 classes.
Found 455 images belonging to 2 classes.


In [45]:
inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(512,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model = tf.keras.Model(inputs, x)

In [58]:
def scheduler(epoch, lr):
  if epoch < 20:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

class mycallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            if(logs['val_accuracy']>=0.998):
                self.model.stop_training=True

In [48]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall()]
              )

In [59]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
tr_callback = mycallback()

model.fit(train_generator,
          epochs=30,
          validation_data = validation_generator,
          callbacks=[lr_callback,tr_callback])

Epoch 1/30
 2/57 [>.............................] - ETA: 9s - loss: 8.5429e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


57/57 [==============================] - 30s 524ms/step - loss: 0.0708 - accuracy: 0.9769 - precision: 0.9707 - recall: 0.9707 - val_loss: 0.1054 - val_accuracy: 0.9714 - val_precision: 0.9560 - val_recall: 0.9721
Epoch 2/30
57/57 [==============================] - 30s 528ms/step - loss: 0.0420 - accuracy: 0.9819 - precision: 0.9777 - recall: 0.9763 - val_loss: 0.1857 - val_accuracy: 0.9516 - val_precision: 0.9937 - val_recall: 0.8827
Epoch 3/30
57/57 [==============================] - 30s 531ms/step - loss: 0.0401 - accuracy: 0.9835 - precision: 0.9818 - recall: 0.9763 - val_loss: 0.1923 - val_accuracy: 0.9538 - val_precision: 0.9817 - val_recall: 0.8994
Epoch 4/30
57/57 [==============================] - 30s 530ms/step - loss: 0.0497 - accuracy: 0.9824 - precision: 0.9804 - recall: 0.9749 - val_loss: 0.1474 - val_accuracy: 0.9516 - val_precision: 0.9067 - val_recall: 0.9777
Epoch 5/30
57/57 [==============================] - 30s 528ms/step - loss: 0.0191 - accuracy: 0.9945 - precisio

In [60]:
model.save("model_v1_resnet50")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_v1_renet50/assets


**Test**

In [ ]:
model = tf.keras.models.load_model('')

In [ ]:
uploaded = files.upload()
for img_path in uploaded.keys():
  img = load_img(img_path, target_size=(299, 299))  # this is a PIL image
  x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
  x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

  # Rescale by 1/255
  x /= 255

  class_ = model.predict(x)
  if class_[0][0] >0.5:
    print("Other")
  else:
    print("")

Saving Screenshot (64).png to Screenshot (64).png
Other


In [ ]:
imagedatagen = ImageDataGenerator(rescale=1/255,
                                  validation_split=0.2,
                            )
datagen = imagedatagen.flow_from_directory(
                            '/content/data',
                            class_mode="binary",
                            target_size=(299,299),
                            batch_size=32)

Found 61 images belonging to 1 classes.


In [ ]:
folder = ''
files = os.listdir(folder)

for img_path in  files:
  if img_path == '.ipynb_checkpoints' or img_path == 'desktop.ini':
    continue
  print(img_path)
  img = load_img(folder+img_path, target_size=(299, 299))  # this is a PIL image
  x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
  x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

  # Rescale by 1/255
  x /= 255

  class_ = model.predict(x)
  if class_[0][0] >0.5:
    print("Other")
  else:
    print("skype")
    # creating a object